# SQL mit Python und Sqlite3
## Proof of Concept - Matthias Ebert, DuP 2022

In [1]:
import sqlite3
#con = sqlite3.connect("Konto.db")   # (neue) sqlite3-Dateien Konto.db und -journal speichern dauerhaft auf Festplatte
con = sqlite3.connect(":memory:")   # Tabellen existieren nur zur Laufzeit im Memory

In [4]:
con.execute("DROP TABLE IF EXISTS Kunde")
con.executescript("""

CREATE TABLE Kunde
(
    KID INT PRIMARY KEY,
    Name CHAR(50)
);

DROP TABLE IF EXISTS Kontobewegungen;

CREATE TABLE Kontobewegungen
(
    KID INT,
    Betrag REAL,
    FOREIGN KEY (KID) REFERENCES Kunde(KID)
);



""")

In [5]:
con.executescript("""

DELETE FROM Kontobewegungen;  
-- Grund: Tabelle löschen, um bei mehrfachen Aufruf nicht Datensätze mehfach einzufügen

INSERT INTO Kontobewegungen (KID,Betrag)
VALUES(1, 123.45),
      (2, 100.00),
      (3, 100.00),
      (1,-100.00),
      (1,  22.00),
      (2,-500.00),
      (3,+500.00);
    
DELETE FROM Kunde;

INSERT INTO Kunde (KID,Name)
VALUES(1, "Anna"),
      (2, "Bert"),
      (3, "Chris"),
      (4, "Dobi");   
""")

In [6]:
result = con.execute("SELECT * FROM Kunde").fetchall()
result

[(1, 'Anna'), (2, 'Bert'), (3, 'Chris'), (4, 'Dobi')]

In [7]:
result = con.execute("""
SELECT KID, SUM(Betrag) AS Bestand
FROM Kontobewegungen
GROUP BY KID
""").fetchall()

result

[(1, 45.45), (2, -400.0), (3, 600.0)]

## Zugriff auf einzelne Elemente
mit Python: list of sets

In [8]:
len(result)

3

In [9]:
for row in result:
    print(row[0], row[1])

1 45.45
2 -400.0
3 600.0


In [10]:
print(result[1][1])

-400.0


In [11]:
for row in result:
    for col in row:
        print(col, end="\t")
    print()

1	45.45	
2	-400.0	
3	600.0	


## Besser mit Pandas
mit schöneren Ausgabetabellen 
Aufgabe: Versuche das Gleiche mit Pandas machen, was du von SQL kennst ;)

In [12]:
import pandas as pd

In [13]:
erg = pd.read_sql("SELECT * FROM Kontobewegungen", con)
erg

,KID,Betrag
0,1,123.45
1,2,100.00
2,3,100.00
3,1,-100.00
4,1,22.00
5,2,-500.00
6,3,500.00


In [ ]:
erg.to_markdown("group.md")
erg.to_html("group.html")

## Group By

In [14]:
pd.read_sql("SELECT KID, SUM(Betrag) AS Bestand FROM Kontobewegungen GROUP BY KID", con)

,KID,Bestand
0,1,45.45
1,2,-400.00
2,3,600.00


In [15]:
erg.groupby("KID").Betrag.sum()

KID
1     45.45
2   -400.00
3    600.00
Name: Betrag, dtype: float64

In [16]:
pd.DataFrame(erg.groupby("KID").Betrag.sum())

,Betrag
KID,
1,45.45
2,-400.00
3,600.00


## SQL Join = Pandas Merge

In [17]:
kunde = pd.read_sql("SELECT * FROM kunde", con)
kunde

,KID,Name
0,1,Anna
1,2,Bert
2,3,Chris
3,4,Dobi


In [18]:
kontostaende = pd.read_sql("SELECT KID, SUM(Betrag) AS Bestand FROM kontobewegungen GROUP BY KID", con)
kontostaende

,KID,Bestand
0,1,45.45
1,2,-400.00
2,3,600.00


In [19]:
pd.read_sql("""
SELECT K.KID, K.Name, SUM(KB.Betrag) AS Bestand 
FROM Kunde K
INNER JOIN Kontobewegungen KB ON K.KID = KB.KID
GROUP BY K.KID
""", con)

,KID,Name,Bestand
0,1,Anna,45.45
1,2,Bert,-400.00
2,3,Chris,600.00


In [20]:
kunde.merge(kontostaende, left_on='KID', right_on='KID')
# vgl. ON Kunde.KID = Kontostaende.KID

,KID,Name,Bestand
0,1,Anna,45.45
1,2,Bert,-400.00
2,3,Chris,600.00


In [21]:
pd.read_sql("""
SELECT K.KID, K.Name, SUM(KB.Betrag) AS Bestand 
FROM Kunde K
LEFT JOIN Kontobewegungen KB ON K.KID = KB.KID
GROUP BY K.KID
""", con)

,KID,Name,Bestand
0,1,Anna,45.45
1,2,Bert,-400.00
2,3,Chris,600.00
3,4,Dobi,NaN


In [22]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde.merge(kontostaende, how="left", left_on='KID', right_on='KID')

,KID,Name,Bestand
0,1,Anna,45.45
1,2,Bert,-400.00
2,3,Chris,600.00
3,4,Dobi,NaN


## Group By Having

In [23]:
pd.read_sql("""
SELECT K.KID, K.Name, SUM(KB.Betrag) AS Bestand 
FROM Kunde K
INNER JOIN Kontobewegungen KB ON K.KID = KB.KID
GROUP BY K.KID
HAVING Bestand >= 0
""", con)

,KID,Name,Bestand
0,1,Anna,45.45
1,3,Chris,600.00


In [24]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde.merge(kontostaende, left_on='KID', right_on='KID')[kontostaende.Bestand >= 0]

,KID,Name,Bestand
0,1,Anna,45.45
2,3,Chris,600.00


## Weitere SQL-Aufgaben in Pandas umsetzen

In [25]:
pd.read_sql("SELECT * FROM kunde WHERE KID = 2", con)

,KID,Name
0,2,Bert


In [26]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde[kunde.KID==2]

,KID,Name
1,2,Bert


In [27]:
pd.read_sql("SELECT * FROM kunde WHERE KID BETWEEN 1 AND 2", con)

,KID,Name
0,1,Anna
1,2,Bert


In [28]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde[kunde.KID.between(1,2)]

,KID,Name
0,1,Anna
1,2,Bert


In [29]:
pd.read_sql("SELECT * FROM kunde WHERE Name BETWEEN 'B' AND 'D'", con)

,KID,Name
0,2,Bert
1,3,Chris


In [30]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde[kunde.Name.between('B','D')]

,KID,Name
1,2,Bert
2,3,Chris


In [31]:
pd.read_sql("SELECT * FROM kunde WHERE Name LIKE 'B%'", con)

,KID,Name
0,2,Bert


In [32]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde[kunde.Name.str.startswith('B')]

,KID,Name
1,2,Bert


In [33]:
pd.read_sql("SELECT * FROM kunde WHERE Name LIKE '%r%'", con)

,KID,Name
0,2,Bert
1,3,Chris


In [34]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde[kunde.Name.str.contains('r')]

,KID,Name
1,2,Bert
2,3,Chris


In [35]:
pd.read_sql("SELECT * FROM kunde WHERE Name IN ('Anna','Chris')", con)

,KID,Name
0,1,Anna
1,3,Chris


In [36]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde.query("Name in ('Anna','Chris')")

,KID,Name
0,1,Anna
2,3,Chris


## Sortieren

In [37]:
pd.read_sql("SELECT * FROM kunde ORDER BY Name DESC", con)

,KID,Name
0,4,Dobi
1,3,Chris
2,2,Bert
3,1,Anna


In [38]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde.sort_values(by='Name', ascending=False)

,KID,Name
3,4,Dobi
2,3,Chris
1,2,Bert
0,1,Anna


# Finde den Kunden mit dem höchsten Kontostand


In [39]:
pd.read_sql("""
  SELECT Name
  FROM 
  (
     SELECT KID, Name, MAX(Bestand)
     FROM 
     (
        SELECT K.KID, K.Name, SUM(KB.Betrag) AS Bestand 
        FROM Kunde K
        INNER JOIN Kontobewegungen KB ON K.KID = KB.KID
        GROUP BY K.KID
     )
  )
""", con)

,Name
0,Chris


In [40]:
# Bitte das Gleiche hier mit Pandas-Mitteln machen
kunde.merge(kontostaende, left_on='KID', right_on='KID')[kontostaende.Bestand == kontostaende.Bestand.max()].Name

2    Chris
Name: Name, dtype: object